In [1]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:
latestsubdir = list(os.walk(path_data))[0][1][-1]
latestdir = path_data + latestsubdir


In [4]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldfAG = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)
dfAG2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
dfAG = fulldfAG.iloc[:-2,:].copy()

# Format columns correctly
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
dfAG["PosPct"] = pd.to_numeric(dfAG["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG["Date"] = dfAG["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    dfAG[name] = pd.to_numeric(dfAG[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))

dfAG2["Dato"] = dfAG2["Dato"].astype('datetime64[D]')
dfAG2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(dfAG2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(dfAG2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

# Cut off early data in PCR dataframe
dfPCR = dfPCR[dfPCR.Date >= dfAG.iloc[0].Date]

# Also cutoff everything before first of January
firstDate = np.datetime64('2021-02-01')
dfPCR = dfPCR[dfPCR.Date >= firstDate]
dfAG = dfAG[dfAG.Date >= firstDate]

# Cut off last date (since it is not fully done)
dfPCR = dfPCR.iloc[:-1]
dfAG = dfAG.iloc[:-1]

# Get dates
curDates = dfPCR.Date.values.astype('datetime64[D]')
confDates = dfAG2.Dato.values.astype('datetime64[D]')

In [5]:
dfAG.tail()
dfPCR.tail()
dfAG2.tail()

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
350,2022-01-17,273730,11766,9882,83.987761,1884,16.012239,8768,833,11458,43104
351,2022-01-18,266106,11832,9786,82.707911,2046,17.292089,8679,883,10487,42754
352,2022-01-19,248133,10733,8616,80.275785,2117,19.724215,7688,759,9383,40385
353,2022-01-20,253772,10436,8400,80.490609,2036,19.509391,7484,757,9153,37280
354,2022-01-21,256877,10508,8176,77.807385,2332,22.192615,7317,717,8451,33407


In [6]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(12,16),tight_layout=True)

PCRpos = dfPCR.NewPositive
AGpos = dfAG.NewPositive
PCRtest = dfPCR.Tested
AGtest = dfAG.Tested
ax1.plot(curDates,PCRtest,label='PCR')
ax1.plot(curDates,AGtest,label='AG')
ax2.plot(curDates,PCRpos,label='PCR')
ax2.plot(curDates,AGpos,label='AG')
ax3.plot(curDates,100 * np.divide(PCRpos,PCRtest),label='PCR')
ax3.plot(curDates,100 * np.divide(AGpos,AGtest),label='AG')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))
# ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))
# ax3.xaxis.set_major_formatter(mdates.DateFormatter('%d. %b'))

ax1.set_ylabel('Antal tests')
ax2.set_ylabel('Antal positive')
ax3.set_ylabel('Positiv procent [%]')


ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)

ax1.legend()
ax2.legend()
ax3.legend()
# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

# ax1.set_xlim([firstDate,lastDate])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
dfAG2.tail()
750*np.power((30000/140000),0.3)

472.4536723764267

In [8]:
# Beregninger
PCRpos = dfPCR.NewPositive.values
PCRtest = dfPCR.Tested.values
AGpos = dfAG.NewPositive.values
AGtest = dfAG.Tested.values
AGposPCRpos = dfAG2.AGpos_PCRpos.values

PCRposPct = np.divide(PCRpos,PCRtest)
AGposPct = np.divide(AGpos,AGtest)

# Calculate the true positive rate
AGtruePos = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos)
avgTruePos = AGtruePos.iloc[-14:].mean()
avgConf = 0.01*dfAG2['Andel_AGpos_PCRkonf_pct'].iloc[-14:].mean()

# Projectér hvor mange der bekræftes i de to dage der endnu ikke er bekræftet
# AGproj = np.divide(avgTruePos*dfAG2['AG_pos'],0.01*dfAG2['Andel_AGpos_PCRkonf_pct'].values)
AGproj = np.divide(avgTruePos*AGpos,avgConf)
confDatesProj = np.concatenate([confDates[:],curDates[-2:]])
confDatesProj = np.append(confDates[:],curDates[-2:]).astype('datetime64[D]')
AGposPCRposProj = np.append(AGposPCRpos,AGproj[-2:])
# 
PCRpos_nonAG = PCRpos - AGposPCRposProj

In [9]:
# fig,(ax1,ax2) = plt.subplots(2,1,tight_layout=True,sharex=True)
fig,(ax3,ax1,ax2) = plt.subplots(3,1,tight_layout=True,sharex=True)


ax3.plot(curDates,PCRpos,label='PCR-positive, alle')
# ax3.plot(curDates,PCRpos_nonAG,label='PCR-positive, uden AG-konfirmatorisk')
ax3.plot(curDates,AGposPCRposProj,'g',label='PCR-positive, AG-konfirmatoriske')
# ax3.set_yscale('log')


ax1.plot(curDates,PCRpos,label='PCR-positive, alle')
# ax1.plot(curDates,PCRpos_nonAG,label='PCR-positive, uden AG-konfirmatorisk')
ax1.plot(curDates,AGposPCRposProj,'g',label='PCR-positive, AG-konfirmatoriske')

ax2.plot(curDates,100*np.divide(AGposPCRposProj,PCRpos),'g',label='AG-konfirmatoriske')

# ax1.plot(dfAG2.Dato,dfAG2.AGpos_PCRkonf)
# ax1.plot(dfPCR.Date,dfPCR.NewPositive)


ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)
# ax2.set_ylim(top=100)
ax3.set_ylim(top=1500)

ax1.set_ylabel('Antal positive')
ax3.set_ylabel('Antal positive')
ax2.set_ylabel('Andel af test [%]')

ax1.legend()
ax2.legend()
ax3.legend(loc='upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig,(ax1,ax2) = plt.subplots(2,1,tight_layout=True,sharex=True)

ag_TP = dfAG2.AGpos_PCRpos
ag_FP = dfAG2.AGposPCRneg
ag_TN = dfAG2.AGnegPCRneg
ag_FN = dfAG2.AGnegPCRpos

ag_sens = np.divide(ag_TP,ag_TP+ag_FN)
ag_spec = np.divide(ag_TN,ag_TN+ag_FP)
ag_prec = np.divide(ag_TP,ag_TP+ag_FP) # Precision, or positive predictive value
ag_accu = np.divide(ag_TP+ag_TN,ag_TP+ag_TN+ag_FP+ag_FN)

ax1.plot(dfAG2.Dato,ag_sens,label='AG sensitivity')
ax2.plot(dfAG2.Dato,ag_spec,label='AG specificity')
ax1.plot(dfAG2.Dato,ag_prec,label='AG Precision')
ax2.plot(dfAG2.Dato,ag_accu,label='AG accuracy')


ax1.legend()
ax2.legend()

# AG_TP = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AGposPCRneg+dfAG2.AGpos_PCRpos)
# AG_P = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AGposPCRneg+dfAG2.AGpos_PCRpos)

# ax1.plot(dfAG2.Dato,dfAG2.Andel_AGpos_PCRkonf_pct)
# ax1.plot(dfAG2.Dato,100*AG_TP)
# ax2.plot(dfAG2.Dato,dfAG2.AG_testede)
# ax2.plot(dfAG2.Dato,dfAG2.AG_pos)
# ax2.plot(dfAG2.Dato,dfAG2.AGpos_PCRkonf)
# ax2.plot(dfAG2.Dato,dfAG2.AGpos_minusPCRkonf)
# ax2.plot(dfAG2.Dato,dfAG2.AGpos_PCRkonf+dfAG2.AGpos_minusPCRkonf,':')
# ax2.plot(dfAG2.Dato,AG_TP)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# fig,ax1 = plt.subplots(1,1,sharex=True)
# meanWidth = 7

# ax1.plot(curDates,AGpos,'.-',color='xkcd:green',linewidth=0.5,markersize=2)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGpos,meanWidth),color='xkcd:green',label='AG positive, total')

# ax1.plot(confDatesProj,AGposPCRposProj,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2)
# ax1.plot(confDatesProj[-2:],AGposPCRposProj[-2:],'*',color='xkcd:dark green',markersize=5)
# ax1.plot(rnTime(confDatesProj,meanWidth),rnMean(AGposPCRposProj,meanWidth),color='xkcd:dark green',label='AG positive, PCR bekræftede')


# ax1.legend(loc='upper left')
# ax1.grid()
# # ax1.set_ylim(bottom=0,top=500)
# ax1.set_ylabel('Antal')

In [12]:
# fig,ax1 = plt.subplots(1,1,sharex=True)

# ax1.plot(curDates,PCRpos,'.-',color='xkcd:brown',linewidth=0.5,markersize=2)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),color='xkcd:brown',label='PCR positive, alle')

# ax1.plot(curDates,AGposPCRposProj,'.-',color='xkcd:dark green',linewidth=0.5,markersize=2)
# ax1.plot(curDates[-2:],AGposPCRposProj[-2:],'*',color='xkcd:dark green',markersize=5)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(AGposPCRposProj,meanWidth),color='xkcd:dark green',label='AG positive, PCR bekræftede')

# ax1.plot(curDates,PCRpos_nonAG,'.-',color='xkcd:dark blue',linewidth=0.5,markersize=2)
# ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color='xkcd:dark blue',label='PCR positive, uden AG bekræftning')

# ax1.legend(loc='upper left')
# ax1.grid()

# Metoder fra rapport

In [13]:
# Beregninger
PCRpos = dfPCR.NewPositive.values
numPCR = dfPCR.Tested.values
AGpos = dfAG.NewPositive.values
numAG = dfAG.Tested.values
AGposPCRpos = dfAG2.AGpos_PCRpos.values

PCRposPct = np.divide(PCRpos,numPCR)
AGposPct = np.divide(AGpos,numAG)

# Calculate the true positive rate
AGtruePos = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos)
avgTruePos = AGtruePos.iloc[-14:].mean()
avgConf = 0.01*dfAG2['Andel_AGpos_PCRkonf_pct'].iloc[-14:].mean()

# Projectér hvor mange der bekræftes i de to dage der endnu ikke er bekræftet
# AGproj = np.divide(avgTruePos*dfAG2['AG_pos'],0.01*dfAG2['Andel_AGpos_PCRkonf_pct'].values)
AGproj = np.divide(avgTruePos*AGpos,avgConf)
confDatesProj = np.concatenate([confDates[:],curDates[-2:]])
confDatesProj = np.append(confDates[:],curDates[-2:]).astype('datetime64[D]')
AGposPCRposProj = np.append(AGposPCRpos,AGproj[-2:])
# 
PCRpos_nonAG = PCRpos - AGposPCRposProj

# 
PCRnoscreen = PCRpos_nonAG * ( (30000/dfPCR.Tested)**0.7)
PCRscreen = PCRpos_nonAG - PCRnoscreen

# Bredde af rullende gennemsnit
meanWidth = 7


In [14]:
fig,ax1 = plt.subplots(1,1,sharex=True,tight_layout=True)


cmap = plt.cm.get_cmap('tab20',7)

ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRnoscreen,meanWidth),color=cmap(0),label='PCR positive, ved 30.000 tests')
ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),rnMean(PCRnoscreen,meanWidth),color=cmap(1),label='PCR positive, kun screening')


ax1.fill_between(rnTime2(curDates,meanWidth),rnMean(PCRpos,meanWidth),rnMean(PCRpos_nonAG,meanWidth),color=cmap(2),label='PCR positive, efter AG positive') 

# ax1.plot(curDates,PCRpos,'k.:',markersize=4,linewidth=0.5)

ax1.legend(loc='upper left')
ax1.grid(color='k')
ax1.set_ylim(bottom=0)
# ax1.set_ylim(bottom=0,top=1100)
ax1.set_ylabel('Antal')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

curLastDate = rnTime2(curDates,meanWidth)[-1]
ax1.set_xlim([firstDate,curLastDate])
# plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18659.0, 19015.0)

In [15]:
AGpospct2 = np.divide(dfAG2['AGpos_PCRpos'],dfAG2['AG_testede'])
AGpospct2Mean = AGpospct2.values[-14:].mean()
AGpospct2 = np.append(AGpospct2,AGpospct2Mean)
AGpospct2 = np.append(AGpospct2,AGpospct2Mean)

AGsens = np.divide(dfAG2['AGpos_PCRpos'], dfAG2['AGpos_PCRpos'] + dfAG2['AGnegPCRpos'])
AGsensmean = AGsens.values[-14:].mean()
AGsens = np.append(AGsens,AGsensmean)
AGsens = np.append(AGsens,AGsensmean)

pcrScreenSimple = dfPCR.Tested - 30000

hardCodedSens = 0.618
# hardCodedSens = 0.492
curEstimate = np.multiply(pcrScreenSimple.values,np.divide(AGpospct2,hardCodedSens))


In [16]:
fig,ax1 = plt.subplots(tight_layout=True)




meanDays = rnTime2(curDates,meanWidth)


ax1.stackplot(meanDays,
    rnMean(PCRpos-AGposPCRposProj - curEstimate,meanWidth),
    rnMean(curEstimate,meanWidth),
    rnMean(AGposPCRposProj,meanWidth),
    labels=['PCR positive, ved 30.000 tests','PCR positive, kun screening','PCR positive, efter AG positive'],
    colors=[cmap(0),cmap(1),cmap(2)]
    )

ax1.legend(loc='upper left')
ax1.grid(color='k')
# ax1.set_ylim(bottom=0,top=7000)
ax1.set_ylim(bottom=0) 
# ax1.set_ylim(bottom=0,top=1100)
ax1.set_ylabel('Antal')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

curLastDate = rnTime2(curDates,meanWidth)[-1]
firstDate = np.datetime64('2021-03-01')
ax1.set_xlim([firstDate,curLastDate])
# plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/Screening_Metode2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18687.0, 19015.0)